In [1]:
import mdtraj as md

In [12]:
# extract backbone atoms and write xyz file

t = md.load('2f4k.pdb')
sel = t.topology.select('backbone')

t_bb = t.restrict_atoms(sel)
t_bb.save_xyz('2f4k_bb.xyz')

In [1]:
import numpy as np
import numba as nb

import folding

In [16]:
@nb.jit(nopython=True)
def calculate_energy(d, d0):
    '''Calculate the energy of the system given the pairwise distance between
    atoms, stored in the (n_atoms, n_atoms) 2D array `d` and the folded reference
    distances stored in an array of the same shape, `d0`.
    
    E = E_bonded + E_nat + E_nb
    '''

    E = (calculate_bonded_energy(d, d0) 
            +  calculate_nativecontact_energy(d, d0) 
            + calculate_nonbonded_energy(d, d0))

    return E


@nb.jit(nopython=True)
def calculate_bonded_energy(d, d0):
    '''Calculate the bond energy between consecutive pairs of atoms
    along the chain. The energy is that of a harmonic spring with rest
    length equal to the distance in the native structure'''

    assert d.shape == d0.shape
    n_atoms = d.shape[0]
    E = 0.0

    for i in range(n_atoms - 1):
        E += (d[i,i+1] - d0[i,i+1])**2

    return E


@nb.jit(nopython=True)
def calculate_nativecontact_energy(d, d0):
    '''Calculate the Go-like energy term that stabilizes native contacts'''

    n_atoms = d.shape[0]
    E = 0.0
    cutoff = 7.0 # units of angstroms

    # loop over all pairs of atoms, (i,j) subject to i < j
    # if d0[i,j] < cutoff then atoms are "in contact" in the native 
    # structure
    for i in range(n_atoms):
        for j in range(i + 1, n_atoms):

            if d0[i,j] <= cutoff:
                sigma = 2.0**(-1.0/6.0) * d0[i,j]
                x = sigma / d[i,j]
                E += 4.0*(x**12 - x**6)

    return E


@nb.jit(nopython=True)
def calculate_nonbonded_energy(d, d0):
    '''Calculate the repulsive interaction between pairs of atoms
    that do not form native contacts'''

    n_atoms = d.shape[0]
    E = 0.0
    native_cutoff = 7.0  # This value is repeated in the nativecontacts terms and should be parameterized
    nb_cutoff = 3.0
    sigma = 2.0**(-1.0/6.0) * nb_cutoff

    for i in range(n_atoms):
        for j in range(i + 1, n_atoms):

            if d0[i,j] > native_cutoff:
                x = sigma / d[i,j]
                if d[i,j] < nb_cutoff:
                    E += 4.0*(x**12 - x**6)

    return E


@nb.jit(nopython=True)
def calculate_distances(coords):
    '''Calculate the pairwise distances between all atoms returning an
    (n_atoms, n_atoms) array, where d_{i,j} is the Euclidean distance
    between atom i and atom j.'''

    n_atoms = coords.shape[0]
    n_dims = coords.shape[1]
    d = np.empty((n_atoms, n_atoms), dtype=np.float64)

    # We will loop over all pairs, but an obvious place for speeding this up
    # is recognizing that d_{i,j} == d_{j,i} and d_{i,i] == 0.0
    for i in range(n_atoms):
        for j in range(n_atoms):
            d_ij = 0.0
            for k in range(n_dims):
                r = coords[i,k] - coords[j,k]
                d_ij += r * r

            d[i,j] = np.sqrt(d_ij)

    return d


@nb.jit(nopython=True)
def generate_trial_move(coords, delta):
    '''Displace all of the atoms in the system by a random
    amount drawn from a normal distribution with std delta
    
    Possible optimization: pass in an pre-allocated array
    for `trial_coords` so we don't keep re-allocating memory.
    
    Could also probably do this with a simple array addition'''

    # make a copy of the coordinates so we don't alter the original
    trial_coords = coords.copy()
    n_atoms, n_dims = coords.shape

    for i in range(n_atoms):
        for j in range(n_dims):
            trial_coords[i,j] += np.random.randn() * delta

    return trial_coords

@nb.jit(nopython=True)
def propagate(coords, d0, delta, n_steps):
    '''Advance the system forward by taking nsteps using a Metropolis Monte Carlo
    acceptance/rejection scheme'''

    # First get the current energy of the system
    d = calculate_distances(coords)
    E_curr = calculate_energy(d, d0)

    n_accepted = 0

    for i in range(n_steps):
        trial_coords = generate_trial_move(coords, delta)
        d_trial = calculate_distances(trial_coords)
        E_trial = calculate_energy(d_trial, d0)

        dE = E_trial - E_curr
        if dE <= 0:
            accept = True
        else:
            rn = np.random.random()
            a = np.exp(-dE)
            if rn <= a:
                accept = True
            else:
                accept = False

        if accept:
            n_accepted += 1
            E_curr = E_trial
            coords[:] = trial_coords
            d[:] = d_trial

    return coords, n_accepted


In [17]:
ref_coords = folding.read_coordinates('2f4k_bb.xyz')
d0 = folding.calculate_distances(ref_coords)
n_atoms = d0.shape[0]

coords = folding.create_extended_chain(n_atoms, 3.0)


In [18]:
%timeit generate_trial_move(coords, 0.03)

The slowest run took 14462.27 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 12.1 µs per loop


In [19]:
%timeit calculate_distances(coords)

The slowest run took 1431.39 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 84.9 µs per loop


In [20]:
d = calculate_distances(coords)

In [21]:
%timeit calculate_energy(d, d0)

The slowest run took 8785.89 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 40.1 µs per loop


In [22]:
%timeit propagate(coords, d0, 0.03, 1)

The slowest run took 1042.42 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 267 µs per loop


In [16]:
n_atoms

140

In [11]:
@nb.njit
def pairwise_python(X):
    M = X.shape[0]
    N = X.shape[1]
    D = np.empty((M, M), dtype=np.float64)
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = np.sqrt(d)
    return D

@nb.jit(nopython=True)
def calculate_distances(coords):
    '''Calculate the pairwise distances between all atoms returning an
    (n_atoms, n_atoms) array, where d_{i,j} is the Euclidean distance
    between atom i and atom j.'''

    n_atoms = coords.shape[0]
    n_dims = coords.shape[1]
    d = np.empty((n_atoms, n_atoms), dtype=np.float64)

    # We will loop over all pairs, but an obvious place for speeding this up
    # is recognizing that d_{i,j} == d_{j,i} and d_{i,i] == 0.0
    for i in range(n_atoms):
        for j in range(n_atoms):
            d_ij = 0.0
            for k in range(n_dims):
                r = coords[i,k] - coords[j,k]
                d_ij += r * r

            d[i,j] = np.sqrt(d_ij)

    return d

In [12]:
%timeit pairwise_python(coords)

The slowest run took 1534.38 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 85.5 µs per loop


In [13]:
%timeit calculate_distances(coords)

The slowest run took 1594.88 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 85 µs per loop


In [14]:
np.allclose(pairwise_python(coords), calculate_distances(coords))

True